<a href="https://colab.research.google.com/github/ms624atyale/Scratch/blob/main/AppliedPhonetics_ModifiedfromMK316.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐹 🐾 🍩 Under Construction: Vocabulary Drills

#📘 **Topic 03 Pronunciation teaching**

**Table of Contents:**  
using **{gTTS}** Text-to-Speech & CMU pronunciation dictionary.  

* Exposure to Keyword pronunciation (using 📍_frequency distribution, gTTS_)
* English rhyming (using 📍_CMU dictionary_): e.g., night, right, bite, etc.
* Learning English vowels with rhyming words.


* [🚫 Not Yet Available] Phontics for adult learners (using 📍_gTTS_: ): Sound letter correspondences

💾 Sample text: [Aesop fable: The Heron](https://raw.githubusercontent.com/MK316/workshop22/main/data/TheHeron.txt) Copy and get it ready to past below :-)

In [ ]:
#@markdown 🔳 Paste the text here for analysis: (text)
text = input()

 A Heron was walking sedately along the bank of a stream, his eyes on the clear water, and his long neck and pointed bill ready to snap up a likely morsel for his breakfast. The clear water swarmed with fish, but Master Heron was hard to please that morning. "No small fry for me," he said. "Such scanty fare is not fit for a Heron. "Now a fine young Perch swam near. "No indeed," said the Heron. "I wouldn't even trouble to open my beak for anything like that!" As the sun rose, the fish left the shallow water near the shore and swam below into the cool depths toward the middle. The Heron saw no more fish, and very glad was he at last to breakfast on a tiny Snail. Do not be too hard to suit or you may have to be content with the worst or with nothing at all.


> 🔲 **Preprocessing of the text**

Set up for processing: no action required.

In [ ]:
#@markdown 🔳 Install and import packages
%%capture
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download("punkt")

!pip install corpus-toolkit

#@markdown 🔳 Create a foloder named "txtdata" for further processing
import os
os.mkdir("txtdata")

#@markdown 🔳 Write text to a file under 'txtdata' folder

with open('txtdata/mytext.txt','w') as f:
  f.write(text)


> 🔲 **Frequency analysis:** your actions required (2 times)

In [ ]:
#@markdown 🔳 Tokenize, getting frequency list with tagging information

from nltk.tokenize import RegexpTokenizer
retokenize =RegexpTokenizer("[\w]+")
words = retokenize.tokenize(text)
words
print('Before stopwords: %d'%len(words))

# Lower case
wlist = []
for w in words:
  w1 = w.lower()
  wlist.append(w1)

words = wlist
#@markdown 🔳 Remove stopwords for frequency distribution analysis:

# import stopwords from nltk.corpus

from nltk.corpus import stopwords
nltk.download("stopwords")

words = [w for w in words if not w in stopwords.words('english')]
print('After stopwords: %d'%len(words))

#@markdown 🔳 POS tagging

from corpus_toolkit import corpus_tools as ct

brown_corp = ct.ldcorpus("txtdata") #load and read text files under 'txtdata' directory
tok_corp = ct.tokenize(brown_corp)  #tokenize corpus - by default this lemmatizes as well
brown_freq = ct.frequency(tok_corp) #creates a frequency dictionary

ct.write_corpus("tagged_txt",ct.tag(ct.ldcorpus("txtdata")))

tagged_freq = ct.frequency(ct.reload("tagged_txt"))
# ct.head(tagged_freq, hits = 10)

#@markdown 🔳 Result saving as a csv file with POS information

import pandas as pd
data_dict = tagged_freq
data_items = data_dict. items()
data_list = list(data_items)
df = pd.DataFrame(data_list)

df.columns=['Tagged_words','Freq']

mycol = list(df['Tagged_words'])

# print(df)

# Word, POS into dataframe

wlist = []
cat = []

for w in mycol:
  w1 = w.split("_")
  wlist.append(w1[0])
  cat.append(w1[1])

df['Word'] = wlist
df['POS'] = cat

#@markdown 🔳 🚩 Sorting by? Answer [pop up box]

print("Sorting by Frequency (type '1'), POS & Freq (type '2'), or by Word alphabetically (type '3')")
sorting = input()

for t in sorting:
  if t == "1":
    df = df.sort_values(by=['Freq'], ascending = False)
  if t == "2":
    df = df.sort_values(by=['POS', 'Freq'], ascending = False)
  if t == "3":
    df = df.sort_values(by=['Word'], ascending = True)
  else:
    print("Type 1, 2, or 3")
df['Index'] = range(1,len(df['POS'])+1)

df = df[["Index", "POS", "Word","Freq"]]
# print df.to_string(index=False)

#@markdown 🔳 🚩 Saving file? Answer [pop up box]

print('Save it as a file? (y/n)')
saving = input()

for s in saving:
  if s == "y":
    with open('pos_wordlist.csv','w') as f:
      df.to_csv(f)
    print('File is saved: pos_wordlist.csv')
  if s == "n":
    print('No file will be saved.')

df.head()


Before stopwords: 154
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
After stopwords: 75
Processing mytext.txt (1 of 1 files)
Processing mytext.txt (1 of 1 files)
Processing 1.txt (1 of 1 files)
Sorting by Frequency (type '1'), POS & Freq (type '2'), or by Word alphabetically (type '3')
3
Save it as a file? (y/n)
y
File is saved: pos_wordlist.csv


,Index,POS,Word,Freq
1,1,PROPN,Heron,5
32,2,PROPN,Master,1
51,3,PROPN,Perch,1
87,4,PROPN,Snail,1
0,5,DET,a,6


## **[1] Generating audio file of word reading**  
Result file => df

In [ ]:
#@markdown 🚩 {gTTS} package installation and import
%%capture
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio

In [ ]:
#@markdown Word reading by gTTS:

#@markdown 🚩 Select word POS: 

word_POS_select = "NOUN" #@param = ["NOUN","VERB","ADJ","ADV","PROPN","ALL"]

wordlist = df[df['POS'] == word_POS_select]
wordlist = wordlist.sort_values(by=['Word'], ascending = True)

collist = list(wordlist['Word'])

print(collist)

#@markdown 🚩 Language to choose: (english, korean, french, spanish)
def tts(mytext):
  text_to_say = mytext

# Step ⓵ Language to choose:
  language_to_choose = "en" #@param ["en", "ko", "fr", 'es']
  # lang = language_to_choose
  language = language_to_choose
  print("Play language accent: %s"%language_to_choose)

  gtts_object = gTTS(text = text_to_say,
                     lang = language,
                    slow = False) 
  
  gtts_object.save("mytext.wav")
  return Audio("mytext.wav")

# join wordlist by adding '!': joining with '.' tend to yield an error of saying two words as one. 
text_to_say = '! '.join(collist)
intro_text = "Okay. I'm going to read a wordlist, so repeat after me."
text_to_say = intro_text + text_to_say
tts(text_to_say)


['bank', 'beak', 'bill', 'breakfast', 'depth', 'eye', 'fare', 'fish', 'fry', 'middle', 'morning', 'morsel', 'neck', 'scanty', 'shore', 'stream', 'sun', 'swam', 'water']
Play language accent: en


##**[2] Rhyming words**

**_Note:_** In the following, you'll get rhyming words from CMU dictionary.  
* [CMU pronounciation dictionary](http://www.speech.cs.cmu.edu/cgi-bin/cmudict)

* [CMU tools](http://www.speech.cs.cmu.edu/tools/lextool.html)


😊 Let's first set up target words for today.

In [ ]:
#@markdown **Step 1:** Install {pronouncing}
%%capture
!pip install pronouncing
!pip install cmudict

import pronouncing
import cmudict

\[Description]: 

* For a target word (your choice), we'll find rhyming words with same number of syllables. _e.g., 'grow' (1 syllable) => The result will show one-syllabled word list (randomly chosen from cmu dictionary)_

* We'll then create an audio file reading the rhyming wordlist.

In [ ]:
#@markdown **Step 2:** 	🚩 Find rhyming words with ____: Note that I find some errors (10% of error rate in pronouncing words.)

print("Write a word to search rhyming words:")
rhyme_with = input()
word = rhyme_with

phones = pronouncing.phones_for_word(word)
syll_count = pronouncing.syllable_count(phones[0])

print("Okay, I'm searching for rhyming words with %s."%word)

result = pronouncing.rhymes(word)

print('How many rhyming words to search? (1~20')
n_words = input()
n_words = int(n_words)

# Among the result, select words with same syllable count

wlist = []
throw = []

def syllcount(x):
  phones = pronouncing.phones_for_word(x)
  n = pronouncing.syllable_count(phones[0])
  return n

for w in result:
  if syllcount(w) == syll_count:
    wlist.append(w)
  else:
    throw.append(w)

# random sample
import random
from random import shuffle

b = random.sample(wlist, n_words)

temp = b[:n_words]

wlist = '. '.join(temp)
intro = "These words rhyme with " + str(rhyme_with) + ". " + "Are you ready to listen?"

wlist = intro + wlist
print("Rhyming words: %s"%wlist)

#@markdown tts() function should be defined above [1]:
tts(wlist)


Write a word to search rhyming words:
eye
Okay, I'm searching for rhyming words with eye.
How many rhyming words to search? (1~20
20
Rhyming words: These words rhyme with eye. Are you ready to listen?spry. wry. why. cai. tye. guy. y. sly. frye. heye. shy. vi. why. aye. flye. skye. bly. jai. try. tsai
Play language accent: en


## **[3]English vowels to learn (Sample 4 vowels with representative simple words)**

piece vs. piss
who'd vs. hood

In [ ]:
#@markdown 🔳 Target words to learn pronunciation:

vowel_in_word = "u_root" #@param ["i_piece", "I_skin", "u_root", "U_good"]

ivowel = ["piece", "need","meet", "seat","see","bean","mean","lead","believe","meal"]
iv = ".  ".join(ivowel)
uvowel = ["root","mood","who","rule","afternoon","food","suit","balloon","cool","moon"]
uv = ".  ".join(uvowel)
Ivowel = ["skin","fit","bit","hit","sit","little","silk","milk","hill","ill"]
Iv = ".  ".join(Ivowel)
Uvowel = ["good","book","cook","would","could","should","wood","wolf","woman","put"]
Uv = ".  ".join(Uvowel)
vlist = {'i_piece': iv, 'I_skin': Iv, 'u_root': uv, 'U_good': Uv}

text_to_say1 = vlist.get(vowel_in_word)
intro = "I'll be saying ten words that share the same vowel. Listen carefully. "

text_to_say = intro + text_to_say1
print(vlist[vowel_in_word])
tts(text_to_say)


root.  mood.  who.  rule.  afternoon.  food.  suit.  balloon.  cool.  moon
Play language accent: en


In [ ]:
# Two vowel in comparison: write two words to compare (e.g., hill, heal)
twowords = input()
tts(twowords)

hill, heal
Play language accent: en


##**[4] Larning English vowels with rhyming words**
### **3.1 9 Monophthongs (simple vowels) and 5 diphthong vowels in English**

* Note: Vowel inventories may differ from one dialect to another. We follow abstractly defined General American English vowel system. (See Ladefoged & Johnson (2017), A course in Phonetics.

### English monophthong vowels: 10 vowels (**The schwa vowel** is excluded here)

![](https://github.com/MK316/workshop22/raw/main/img/englishvowels1.png)

In [ ]:
#@markdown **Step 1:** Install {pronouncing}
%%capture
!pip install pronouncing
!pip install gTTS

from gtts import gTTS
from IPython.display import Audio
import pronouncing

In [ ]:
#@markdown **Step2:** 🚩 Vowels to learn:

#@markdown 🐳 Note: 'taught' has the same vowel with 'top' in the speaker's dialect.
Target_vowel_as_in = "bean" #@param ["bean","bin","Ben","ban","mood","would","taught","nod", "mud","go","how","bay","bye","boy"]

word = Target_vowel_as_in

#@markdown 🐳 Number of syllable: 1~3 (Select "ignore" for not using this option.)
number_of_syllables = "1" #@param ["1","2","3","4","ignore"]

# phones = pronouncing.phones_for_word(word)
# syll_count = pronouncing.syllable_count(phones[0])

result = pronouncing.rhymes(word)

# syllable number counting as function
def syllcount(x):
  phones = pronouncing.phones_for_word(x)
  n = pronouncing.syllable_count(phones[0])
  return n

wlist = []

# (syllcount(w) == 1 or 2)
for w in result:
  if (number_of_syllables == "ignore"):
    wlist.append(w)
  elif (int(number_of_syllables) == syllcount(w)) and (len(w) > 1):
    wlist.append(w)
  else:
    throw.append(w)

print("Rhyming words: %d"%len(wlist))

How_many_words_to_show = "10" #@param = ["5","10","15","20","30"]
n_words = int(How_many_words_to_show)
# random sample
import random
from random import shuffle

if len(wlist) > n_words:
  b = random.sample(wlist, n_words)
  temp = b[:n_words]
else:
  temp = wlist


# wordlist as dataframe to display
import pandas as pd
dft = pd.DataFrame()
dft['Words'] = temp

wlist = '. '.join(temp)
intro = "These are randomly chosen " + str(n_words) + " words that rhyme with " + str(word) + ". " + "Are you ready to listen? "

wlist = intro + wlist
print("Rhyming words: \n %s"%wlist)
print("*** Target vowel in '%s'"%word)
print(dft)

tts(wlist)


Rhyming words: 79
Rhyming words: 
 These are randomly chosen 10 words that rhyme with bean. Are you ready to listen? lene. clean. keehn. keene. drouin. lien. shean. deane. bein. peine
*** Target vowel in 'bean'
    Words
0    lene
1   clean
2   keehn
3   keene
4  drouin
5    lien
6   shean
7   deane
8    bein
9   peine
Play language accent: en


In [ ]:
#@markdown Do you want to see the whole rhyming words?
print(result)

['achene', 'agin', 'aileen', 'ailene', 'aladeen', 'aldin', 'aleen', 'alene', 'alzene', 'ameen', 'amin', 'amine', 'ammeen', 'andreen', 'aquamarine', 'ardeen', 'arleen', 'arreguin', 'athene', 'augustin', 'baleen', 'barentine', 'beane', 'beene', 'bein', 'beine', 'benyamin', 'benzene', 'bernadine', 'between', 'between', 'biehn', 'bien', 'blouin', 'bodenstein', 'bornstein', 'brackeen', 'bradeen', 'brean', 'breen', 'brigantine', 'burdine', 'burkeen', 'caffeine', 'canteen', 'cantin', 'careen', 'carleen', 'casein', 'cashion', 'casteen', 'cathleen', 'celine', 'charleen', 'charlene', 'charline', 'chasteen', 'cheane', 'christine', 'cian', 'circumvene', 'citrine', 'claudine', 'clean', 'clymene', 'coffeen', 'coleen', 'colleen', 'convene', 'corinne', 'cortine', 'crean', 'crimean', 'cuisine', 'cunneen', "d'alene", 'dean', 'deane', 'deen', 'dejean', 'demean', 'deneen', 'derouin', 'dineen', 'dinneen', 'doralin', 'doreen', 'drouin', 'dufrene', 'eighteen', 'eighteen', 'eileen', 'etheljean', 'eugene', 'fe

In [ ]:
#@markdown 🐳 Find stress pattern: 
wordstress = input()
pronouncing.stresses(pronouncing.phones_for_word(wordstress)[0])

regulation


'2010'

## [4] (To be updated) Phonics for adult learners: Letter-Sound correspondences